In [1]:
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from future.utils import iteritems
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pymysql
from gensim.models import word2vec
from tqdm.notebook import tqdm

In [8]:
article_data = pd.read_csv('mixver9.txt', encoding='utf-8', header= None)
documents = [' '.join(i[0].split(' ')[1:]) for i in article_data.values]

as_one = ''
for document in documents:
    as_one = as_one + ' ' + document
words = as_one.split()

counts = Counter(words)

vocab = sorted(counts, key=counts.get, reverse=True)

word2idx = {word.encode("utf8").decode("utf8"): ii for ii, word in enumerate(vocab,1)}

idx2word = {ii: word for ii, word in enumerate(vocab)}

V = len(word2idx)
N = len(documents)

tf = CountVectorizer()

tf.fit_transform(documents)

tf.fit_transform(documents)[0:1].toarray()

tfidf = TfidfVectorizer(max_features = 1000, max_df=1, min_df=0)
#generate tf-idf term-document matrix
A_tfidf_sp = tfidf.fit_transform(documents)  #size D x V

tfidf_dict = tfidf.get_feature_names()
tfidf_dict

['가격',
 '가구',
 '가득',
 '가루',
 '가방',
 '가슴',
 '가요',
 '가위',
 '가을',
 '가이드',
 '가이드북',
 '가장',
 '가족',
 '가죽',
 '가지',
 '가치',
 '각인',
 '간식',
 '간편',
 '간호',
 '갈비',
 '감각',
 '감귤',
 '감동',
 '감성',
 '감정',
 '강사',
 '강아지',
 '개념',
 '개발',
 '개선',
 '거리',
 '거울',
 '거치',
 '거품',
 '걱정',
 '건강',
 '건조',
 '건축',
 '걸이',
 '검증',
 '게임',
 '겨울',
 '겨울나기',
 '결혼',
 '경량',
 '경력',
 '경험',
 '계란',
 '계절',
 '고급',
 '고기',
 '고래',
 '고민',
 '고양이',
 '골드',
 '공간',
 '공감',
 '공개',
 '공기',
 '공룡',
 '공부',
 '공식',
 '공연',
 '공용',
 '공존',
 '관리',
 '광고',
 '괴물',
 '교육',
 '교체',
 '구독',
 '구두',
 '구름',
 '구리',
 '구성',
 '구스',
 '국내',
 '국민',
 '국산',
 '귀걸이',
 '규어',
 '그녀',
 '그대',
 '그린',
 '그림',
 '그림책',
 '기념',
 '기능',
 '기록',
 '기름',
 '기모',
 '기본',
 '기부',
 '기분',
 '기술',
 '기억',
 '기적',
 '기준',
 '기타',
 '기획',
 '김치',
 '깔창',
 '꿀잠',
 '나노',
 '나무',
 '나이트',
 '남녀',
 '남성',
 '남자',
 '낭만',
 '내내',
 '냄새',
 '네이버웹툰',
 '노래',
 '노트',
 '노트북',
 '노하우',
 '놀이',
 '높이',
 '누리',
 '느낌',
 '니트',
 '다른',
 '다리',
 '다운',
 '다이아',
 '다이아몬드',
 '다이어리',
 '다이어트',
 '다크',
 '닥터',
 '단독',
 '단백질',
 '단편',
 '단편영화',
 '단행본',
 '달달',
 '달력',


In [9]:
import sys
if sys.version_info <= (2,7):
    reload(sys)
    sys.setdefaultencoding('utf-8')
import konlpy
from konlpy.tag import Kkma, Okt, Hannanum

kkma = Kkma()
okt = Okt()
#hannanum = Hannanum()
print('konlpy version = %s' % konlpy.__version__)

konlpy version = 0.5.2


In [10]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='wdta2181',db='test', charset='utf8')

curs = conn.cursor()

username = "우주정복자"
sql = "select distinct trim(I.title) from test.user_info I, test.crawl C where username like '%s' and C.title = I.title;"%username

curs.execute(sql)
sent = curs.fetchall()

curs.execute(sql)
title_with_category = curs.fetchall()

print("========== title ==========")
print(title_with_category)
print("========== title ==========\n")

noun_set = set(okt.nouns(str(sent)))
selected_word = []

sent_list = list(sent)
sent_list2 = []
for i in sent_list:
    sent_list2.append(list(i))

print("======= selected_word =======")
#selected_word : tokenized word from user's funding title list
for i in noun_set:
    if i in tfidf_dict:
        selected_word.append(i)
        continue
print(selected_word) 
print("======= selected_word =======\n")

final_word_list = []
category_list=[]

#final_word_list : selected words
print("======= category list =======")
for i in selected_word:
    sql3 = "select distinct I.title, C.category from test.user_info I, test.crawl C where username like '{username}' and C.title = I.title and I.title like '%%{word}%%';".format(username=username, word=i);
    curs.execute(sql3)
    rows = curs.fetchall()
    
    for j in rows:
        if i in j[0]:
            final_word_list.append(i)
            category_list.append(j[1])        
print(category_list)
print("======= category list =======\n")  

model = word2vec.Word2Vec.load("NaverMovie29.model")
word_len = len(final_word_list)
category_len = len(category_list)

final_projects=[]
similar_word_list=[]

for i in tqdm(range(word_len)):
    print("key word: "+final_word_list[i], category_list[i])
    print()
    similar_word=model.wv.most_similar(positive=[final_word_list[i]],topn=5)
    for j in similar_word:
        if len(j[0])==1:
            continue
        else:
            print(j[0])
            sql = "select DISTINCT pagename, category, trim(title) from test.crawl where title like '%%%s%%'" %j[0]
            curs.execute(sql)
            words = curs.fetchall()
            conn.commit()
            #print(words)
            length = len(words)
            if length>3:
                length = 3
            #print(length)
            for k in range(0,length):
                tmp=[]
                tmp.append(''.join(list(words[k][2])))
                if words[k][1] == category_list[i] and tmp not in sent_list2 :
                    final_projects.append(words[k])
        print()   


final_projects_set = set(final_projects)
for i in final_projects_set:
    print(i)
print(len(final_projects_set))
conn.close()

========== title ==========
(('고양이 해적 패키지 에코백 뱃지 달력',),)
========== title ==========

======= selected_word =======
['에코', '달력', '고양이', '뱃지', '패키지']
======= selected_word =======

======= category list =======
['디자인', '디자인', '디자인', '디자인', '디자인']
======= category list =======



  0%|          | 0/5 [00:00<?, ?it/s]

key word: 에코 디자인

프린

잔인함

세월

피플

key word: 달력 디자인

캘린더

색연필

탁상

하루하루

팝업

key word: 고양이 디자인

노블

샌드

집사

첨성대

key word: 뱃지 디자인

코숏

이중섭

봄날

경첩

key word: 패키지 디자인

신축년

담화

왕국

심청

('tumblbug', '디자인', 'f o l d r i p. 종이 팝업 드리퍼')
('wadiz', '디자인', 'Vase Clock 인테리어용 아크릴 탁상시계')
('wadiz', '디자인', '하루하루가 소중한 당신에게')
('tumblbug', '디자인', '하루하루 내 운을 알아가는 달력')
('wadiz', '디자인', '카우카우 2021 캘린더는 꼬마공룡 카우와 함께')
('wadiz', '디자인', '공간과 시간을 의미있게 만들다 2021 미닝풀 캘린더')
('wadiz', '디자인', '드로린의 패션피플 커플 에코백')
('wadiz', '디자인', '깨끗한 바다를 위한 2021 바다동물 일러스트 팝업 달력 엽서')
('tumblbug', '디자인', '2021 토끼 색연필 일러스트 엽서 달력 포스터 달력')
('tumblbug', '디자인', '의미 없이 흘러가는 시간은 없어 채우는 달력 허송세월')
('tumblbug', '디자인', '봄날의 서커스 입체 컬러링 아트 할핀 인형')
('wadiz', '디자인', '꿈을 이루는 캘린더 2021 패브릭 캘린더')
('tumblbug', '디자인', '2021년 빨간고래의 탁상 캘린더 색연필 푸드 일러스트')
('wadiz', '디자인', '2021년 업무 필수템 가지고 다니는 탁상달력 포터블 캘린더!')
('tumblbug', '디자인', '힐링되는 하루하루 달력엽서와 함께 하세요')
('wadiz', '디자인', '책상 위 과수원 부자. 사과 일러스트 2021탁상 달력+데꾸테리어 set')
('tumblbug', '디자인', '색연필처럼 포근한 교실 2021 해sun 

In [5]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='wdta2181',db='test', charset='utf8')

curs = conn.cursor()

username = "하늘마당"
sql = "select distinct trim(I.title) from test.user_info I, test.crawl C where username like '%s' and C.title = I.title;"%username

curs.execute(sql)
sent = curs.fetchall()

curs.execute(sql)
title_with_category = curs.fetchall()

print("========== title ==========")
print(title_with_category)
print("========== title ==========\n")

noun_set = set(okt.nouns(str(sent)))
selected_word = []

sent_list = list(sent)
sent_list2 = []
for i in sent_list:
    sent_list2.append(list(i))

print("======= selected_word =======")
#selected_word : tokenized word from user's funding title list
for i in noun_set:
    if i in tfidf_dict:
        selected_word.append(i)
        continue
print(selected_word) 
print("======= selected_word =======\n")

final_word_list = []
category_list=[]

#final_word_list : selected words
print("======= category list =======")
for i in selected_word:
    sql3 = "select distinct I.title, C.category from test.user_info I, test.crawl C where username like '{username}' and C.title = I.title and I.title like '%%{word}%%';".format(username=username, word=i);
    curs.execute(sql3)
    rows = curs.fetchall()
    
    for j in rows:
        if i in j[0]:
            final_word_list.append(i)
            category_list.append(j[1])        
print(category_list)
print("======= category list =======\n")  

model = word2vec.Word2Vec.load("NaverMovie29.model")
word_len = len(final_word_list)
category_len = len(category_list)

final_projects=[]
similar_word_list=[]

for i in tqdm(range(word_len)):
    print("key word: "+final_word_list[i], category_list[i])
    print()
    similar_word=model.wv.most_similar(positive=[final_word_list[i]],topn=5)
    for j in similar_word:
        if len(j[0])==1:
            continue
        else:
            print(j[0])
            sql = "select DISTINCT pagename, category, trim(title) from test.crawl where title like '%%%s%%'" %j[0]
            curs.execute(sql)
            words = curs.fetchall()
            conn.commit()
            #print(words)
            length = len(words)
            if length>3:
                length = 3
            #print(length)
            for k in range(0,length):
                tmp=[]
                tmp.append(''.join(list(words[k][2])))
                if words[k][1] == category_list[i] and tmp not in sent_list2 :
                    final_projects.append(words[k])
        print()   


final_projects_set = set(final_projects)
#for i in final_projects_set:
#    print(i)
#print(final_projects_set)

final_projects_set = list(final_projects_set)
fianl_projects_set = final_projects_set.sort(key = lambda x: [x[0], x[1]])
print()
for i in final_projects_set:
    print(i)
print(len(final_projects_set))

conn.close()

========== title ==========
(('당장 밥말아 먹고 싶은 파스타가 왔다! 데이루나 바지락 국물 파스타',), ('강화도 청소년들이 전하는 섬의 이야기가 담긴 고구마를 요리로 즐겨보세요!',), ('고급향수야? 룸스프레이야? 전세계 고급호텔 향 3만원대 한병에 담다!',), ('7일 한정크리스마스 할머니의 산나물을 담은 오감만족 크리스마스 만찬',), ('집콕필수템 NO설탕 NO물엿 상큼감귤 프리바이오틱 비타민 젤리',), ('모공 연구 10년 흡수 노하우 스위스 검증 비타민C 화장품',), ('7일 단골특집 펀딩요청쇄도! 모공쫙~맑고 탱탱한 피부변화오메가믹스',), ('완판신화 돌아왔다! 김은 먹고 다니냐? 명품 해남 곱창김 100',), ('단 한 권에 열 권을 담은 당신을 위한 디지털 다이어리 SOK',), ('뻔한 음식 말고 간편한 장어로! 세상 맛있는 꿀장어 홈키트',), ('누적 구매평 7천건 찐리뷰가 인정하는 찐 춘천 닭갈비를 간편하게!',), ('200개 한정 제주 매거진 인iiin과 제주 로컬푸드 정기배송!',), ('와인전문가가 추천하는 셰프소믈리에 캐링백+와인잔 컬렉션',), ('입 냄새나 숨 쉬지마! 이제 모과글로 숨 크게 쉬자!',), ('업그레이드 앵콜 다시한번 가격파괴 차량용 무선충전 거치대 핸디그랩',), ('즐거운 요리의 시작 상하좌우 내부까지 ALL 티타늄 도마',), ('45년 전통 3대째 내려오는 비법 된장콩잎',), ('글로벌 26억펀딩 1만원대 6in1만능케이블 inCharge6',))
========== title ==========

======= selected_word =======
['모공', '검증', '인정', '고급', '전문가', '할머니', '컬렉션', '화장품', '충전', '세상', '향수', '비타민', '신화', '명품', '차량', '시작', '전세계', '거치', '셰프', '설탕', '비법', '다이어리', '전통', '로컬', '추천', '연구', '정기', '피부', 

  0%|          | 0/54 [00:00<?, ?it/s]

key word: 모공 뷰티

사수

피지

노폐물

각질

미생물

key word: 모공 뷰티

사수

피지

노폐물

각질

미생물

key word: 검증 뷰티

갈바

백신

어쩌

증가

후기

key word: 인정 푸드

드라이버

서브

메인

듀얼

스펙

key word: 고급 홈리빙

식기

원목

식탁

키핑

호텔

key word: 전문가 홈리빙

한국인

자주

왕초보

특허성

자시

key word: 할머니 푸드

손녀

선생님

하라

지옥

그때

key word: 컬렉션 홈리빙

농부

감칠맛

애니메이션

직송

임명

key word: 화장품 뷰티

체험판

나노입자

엑소

포렌

도화지

key word: 충전 테크

무선

시어스

충전기

뱅크

key word: 세상 푸드

군밤

수가

영문

로서

key word: 향수 홈리빙

압생트

향기

르파써

퍼퓸

바닐라

key word: 비타민 푸드

항산화

사표

잔틴

프로폴리스

히알루론산

key word: 비타민 뷰티

항산화

사표

잔틴

프로폴리스

히알루론산

key word: 신화 푸드

그리스

처마

완판

스텐팬

key word: 명품 푸드

근본

브랜드

테이

퍼드

수페르가

key word: 차량 테크

보냉컵홀더

방향

트라이던트

와이퍼

발상

key word: 시작 홈리빙

금물

핵심

기구

자리

체지방

key word: 전세계 홈리빙

오토

독자

돈버는법

유튜브

key word: 거치 테크

충전기

태블릿

드라이기

스마트폰

모빌리티

key word: 셰프 홈리빙

힐리

파김치

언스

순삭

소믈리에

key word: 설탕 푸드

슈카

대체

탄산음료

상큼

청포도

key word: 비법 푸드

임원경제지

남현

뽀개기

카라멜

유자

key word: 다이어리 디자인

애뽈

캡처

파크

만년

여섯

key word: 전통 푸드

소반

현대

호담

key word: 

In [11]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='wdta2181',db='test', charset='utf8')

curs = conn.cursor()

username = "오아시스사이다"
sql = "select distinct trim(I.title) from test.user_info I, test.crawl C where username like '%s' and C.title = I.title;"%username

curs.execute(sql)
sent = curs.fetchall()

curs.execute(sql)
title_with_category = curs.fetchall()

print("========== title ==========")
print(title_with_category)
print("========== title ==========\n")

noun_set = set(okt.nouns(str(sent)))
selected_word = []

sent_list = list(sent)
sent_list2 = []
for i in sent_list:
    sent_list2.append(list(i))

print("======= selected_word =======")
#selected_word : tokenized word from user's funding title list
for i in noun_set:
    if i in tfidf_dict:
        selected_word.append(i)
        continue
print(selected_word) 
print("======= selected_word =======\n")

final_word_list = []
category_list=[]

#final_word_list : selected words
print("======= category list =======")
for i in selected_word:
    sql3 = "select distinct I.title, C.category from test.user_info I, test.crawl C where username like '{username}' and C.title = I.title and I.title like '%%{word}%%';".format(username=username, word=i);
    curs.execute(sql3)
    rows = curs.fetchall()
    
    for j in rows:
        if i in j[0]:
            final_word_list.append(i)
            category_list.append(j[1])        
print(category_list)
print("======= category list =======\n")  

model = word2vec.Word2Vec.load("NaverMovie29.model")
word_len = len(final_word_list)
category_len = len(category_list)

final_projects=[]
similar_word_list=[]

for i in tqdm(range(word_len)):
    print("key word: "+final_word_list[i], category_list[i])
    print()
    similar_word=model.wv.most_similar(positive=[final_word_list[i]],topn=5)
    for j in similar_word:
        if len(j[0])==1:
            continue
        else:
            print(j[0])
            sql = "select DISTINCT pagename, category, trim(title) from test.crawl where title like '%%%s%%'" %j[0]
            curs.execute(sql)
            words = curs.fetchall()
            conn.commit()
            #print(words)
            length = len(words)
            if length>3:
                length = 3
            #print(length)
            for k in range(0,length):
                tmp=[]
                tmp.append(''.join(list(words[k][2])))
                if words[k][1] == category_list[i] and tmp not in sent_list2 :
                    final_projects.append(words[k])
        print()   


final_projects_set = set(final_projects)
#for i in final_projects_set:
#    print(i)
#print(final_projects_set)

final_projects_set = list(final_projects_set)
fianl_projects_set = final_projects_set.sort(key = lambda x: [x[0], x[1]])
print()
for i in final_projects_set:
    print(i)
print('개수 : ', len(final_projects_set))

conn.close()

========== title ==========
(('스케치업 웹툰 배경_지하철역.버스.학교.놀이터.주택가',), ('스케치업 광역 전망의 마을과 업그레이드 소품 자연물',))
========== title ==========

======= selected_word =======
['웹툰', '마을', '배경', '스케치업', '소품', '자연', '학교']
======= selected_word =======

======= category list =======
['만화', '만화', '만화', '만화', '만화', '만화', '만화', '만화']
======= category list =======



  0%|          | 0/8 [00:00<?, ?it/s]

key word: 웹툰 만화

스케치업

배경

테헤란로

왕궁

해안

key word: 마을 만화

전망

해안

왕궁

외관

토닥

key word: 배경 만화

스케치업

웹툰

무협

사극

왕궁

key word: 스케치업 만화

배경

웹툰

왕궁

중세

해안

key word: 스케치업 만화

배경

웹툰

왕궁

중세

해안

key word: 소품 만화

다이노

돼지

공룡

아기

key word: 자연 만화

도심

냥신

캣폴

올레

key word: 학교 만화

학과

궁전

졸업

성폭력


('tumblbug', '만화', '스케치업 해안마을과 소방차 이삿짐차')
('tumblbug', '만화', '웹툰스케치업배경동양환타지무협사극문파')
('tumblbug', '만화', '웹툰판타지 대형도서관 외관+실내6종')
('tumblbug', '만화', '웹툰 배경 스케치업_중세 다양한 궁전 정원')
('tumblbug', '만화', '세렌 웹툰 스케치업7 테헤란로 도시거리상점가')
('tumblbug', '만화', '스케치업 웹툰 배경역삼 테헤란로')
('tumblbug', '만화', '웹툰사계절 유럽판타지마을 외관+실내3종')
('tumblbug', '만화', '스케치업 전망 좋은 산복도로 마을')
('tumblbug', '만화', '투닥토닥 판타지 스케치업 프로젝트 No.19_ 사막마을')
('tumblbug', '만화', '투닥토닥 판타지 스케치업 프로젝트 No.17_ 휴식')
('tumblbug', '만화', '스케치업 웹툰배경 귀족클럽과 집무실')
('tumblbug', '만화', '웹툰 배경 스케치업_중세 판타지 학교')
('tumblbug', '만화', '스케치업 웹툰 배경 19세기 사용인의 공간 9종세트')
('tumblbug', '만화', '앤트 웹툰 스케치업 1  중세 저택내부')
('tumblbug', '만화', '스케치업 웹툰배경 로판 왕궁 배경')
('tumblbug', '만화', '중세 판타지 왕도대도시')

In [6]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='wdta2181',db='test', charset='utf8')

curs = conn.cursor()

username = "담배요정타르"
sql = "select distinct trim(I.title) from test.user_info I, test.crawl C where username like '%s' and C.title = I.title;"%username

curs.execute(sql)
sent = curs.fetchall()

curs.execute(sql)
title_with_category = curs.fetchall()

print("========== title ==========")
print(title_with_category)
print("========== title ==========\n")

noun_set = set(okt.nouns(str(sent)))
selected_word = []

sent_list = list(sent)
sent_list2 = []
for i in sent_list:
    sent_list2.append(list(i))

print("======= selected_word =======")
#selected_word : tokenized word from user's funding title list
for i in noun_set:
    if i in tfidf_dict:
        selected_word.append(i)
        continue
print(selected_word) 
print("======= selected_word =======\n")

final_word_list = []
category_list=[]

#final_word_list : selected words
print("======= category list =======")
for i in selected_word:
    sql3 = "select distinct I.title, C.category from test.user_info I, test.crawl C where username like '{username}' and C.title = I.title and I.title like '%%{word}%%';".format(username=username, word=i);
    curs.execute(sql3)
    rows = curs.fetchall()
    
    for j in rows:
        if i in j[0]:
            final_word_list.append(i)
            category_list.append(j[1])        
print(category_list)
print("======= category list =======\n")  

model = word2vec.Word2Vec.load("NaverMovie29.model")
word_len = len(final_word_list)
category_len = len(category_list)

final_projects=[]
similar_word_list=[]

for i in tqdm(range(word_len)):
    print("key word: "+final_word_list[i], category_list[i])
    print()
    similar_word=model.wv.most_similar(positive=[final_word_list[i]],topn=5)
    for j in similar_word:
        if len(j[0])==1:
            continue
        else:
            print(j[0])
            sql = "select DISTINCT pagename, category, trim(title) from test.crawl where title like '%%%s%%'" %j[0]
            curs.execute(sql)
            words = curs.fetchall()
            conn.commit()
            #print(words)
            length = len(words)
            if length>3:
                length = 3
            #print(length)
            for k in range(0,length):
                tmp=[]
                tmp.append(''.join(list(words[k][2])))
                if words[k][1] == category_list[i] and tmp not in sent_list2 :
                    final_projects.append(words[k])
        print()   


final_projects_set = set(final_projects)
#for i in final_projects_set:
#    print(i)
#print(final_projects_set)

final_projects_set = list(final_projects_set)
fianl_projects_set = final_projects_set.sort(key = lambda x: [x[0], x[1]])
print()
for i in final_projects_set:
    print(i)
print('개수 : ', len(final_projects_set))

conn.close()

========== title ==========
(('10억 펀딩 신소재 베타젤 다리베개로 다리 붓기다리 케어 다리힐링',), ('4차앵콜 오직 잠에만 집중 잠의 신 기절수면안대',))
========== title ==========

======= selected_word =======
['수면', '베타', '신소재', '케어', '안대', '힐링', '다리', '집중', '베개']
======= selected_word =======

======= category list =======
['홈리빙', '홈리빙', '홈리빙', '홈리빙', '홈리빙', '홈리빙', '홈리빙', '홈리빙', '홈리빙']
======= category list =======



  0%|          | 0/9 [00:00<?, ?it/s]

key word: 수면 홈리빙

임상시험

포렌

안대

기절

숙면

key word: 베타 홈리빙

신소재

붓기

이미드

에테르

key word: 신소재 홈리빙

에테르

베타

이미드

폴리

붓기

key word: 케어 홈리빙

증진

기미

붓기

key word: 안대 홈리빙

포렌

기절

호강

숙면

수면

key word: 힐링 홈리빙

아이콘

익명

만다라

리추얼

불멍

key word: 다리 홈리빙

붓기

베개

베타

진동

key word: 집중 홈리빙

번호판

불교

기절

key word: 베개 홈리빙

수건

다리

붓기

어깨


('wadiz', '홈리빙', '3차 앵콜 1만원대 눈 호강! 핑크소금 찜질 안대')
('wadiz', '홈리빙', '1만원대독일 숙면템 포렌이 만든 화장품을 먹지 않는 수면 안대?!')
('wadiz', '홈리빙', '933앵콜 이제 수건 깔고 자는건 그만! 초간단 신개념 위생베개시트')
('wadiz', '홈리빙', '구름에 머리를 베는 것 같은 완전한 포근함! 구름날개 베개로 경험하세요.')
('wadiz', '홈리빙', '3차앵콜 오직 잠에만 집중 잠의 신 기절 수면 안대')
('wadiz', '홈리빙', '아직도 수건 빨아서 쓰세요? 수건 한 번 하세요!')
('wadiz', '홈리빙', '300만개 판매 베개 두피열을 잡다! 양면으로 사용하는 기절베개 블루')
('wadiz', '홈리빙', '숙면+휴식+진동 마사지를 동시에 피로회복 끝판템 힐링미 바디필로우')
('wadiz', '홈리빙', '매일 어깨 마사지 받는 것처럼!! 목어깨가 가벼워지는 IU어깨베개!!')
개수 :  9


In [13]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='wdta2181',db='test', charset='utf8')

curs = conn.cursor()

username = "박철수"
sql = "select distinct trim(I.title) from test.user_info I, test.crawl C where username like '%s' and C.title = I.title;"%username

curs.execute(sql)
sent = curs.fetchall()

curs.execute(sql)
title_with_category = curs.fetchall()

print("========== title ==========")
print(title_with_category)
print("========== title ==========\n")

noun_set = set(okt.nouns(str(sent)))
selected_word = []

sent_list = list(sent)
sent_list2 = []
for i in sent_list:
    sent_list2.append(list(i))

print("======= selected_word =======")
#selected_word : tokenized word from user's funding title list
for i in noun_set:
    if i in tfidf_dict:
        selected_word.append(i)
        continue
print(selected_word) 
print("======= selected_word =======\n")

final_word_list = []
category_list=[]

#final_word_list : selected words
print("======= category list =======")
for i in selected_word:
    sql3 = "select distinct I.title, C.category from test.user_info I, test.crawl C where username like '{username}' and C.title = I.title and I.title like '%%{word}%%';".format(username=username, word=i);
    curs.execute(sql3)
    rows = curs.fetchall()
    
    for j in rows:
        if i in j[0]:
            final_word_list.append(i)
            category_list.append(j[1])        
print(category_list)
print("======= category list =======\n")  

model = word2vec.Word2Vec.load("NaverMovie29.model")
word_len = len(final_word_list)
category_len = len(category_list)

final_projects=[]
similar_word_list=[]

for i in tqdm(range(word_len)):
    print("key word: "+final_word_list[i], category_list[i])
    print()
    similar_word=model.wv.most_similar(positive=[final_word_list[i]],topn=5)
    for j in similar_word:
        if len(j[0])==1:
            continue
        else:
            print(j[0])
            sql = "select DISTINCT pagename, category, trim(title) from test.crawl where title like '%%%s%%'" %j[0]
            curs.execute(sql)
            words = curs.fetchall()
            conn.commit()
            #print(words)
            length = len(words)
            if length>3:
                length = 3
            #print(length)
            for k in range(0,length):
                tmp=[]
                tmp.append(''.join(list(words[k][2])))
                if words[k][1] == category_list[i] and tmp not in sent_list2 :
                    final_projects.append(words[k])
        print()   


final_projects_set = set(final_projects)
#for i in final_projects_set:
#    print(i)
#print(final_projects_set)

final_projects_set = list(final_projects_set)
fianl_projects_set = final_projects_set.sort(key = lambda x: [x[0], x[1]])
print()
for i in final_projects_set:
    print(i)
print('개수 : ', len(final_projects_set))

conn.close()

========== title ==========
(('4.6평점 1만원대 기모슬랙스 앵콜바지 전문공장이 자신있게 선보입니다.',), ('7일 단골특집 펀딩요청쇄도! 모공쫙~맑고 탱탱한 피부변화오메가믹스',), ('2만 원대 바지맞아? 캠핑부터 운동 데일리룩까지 다 되는 활력팬츠',), ('4채널끝판왕 제대로 만든 블랙박스 보이지 않는 문콕도 두렵지 않다!',), ('글로벌 26억펀딩 1만원대 6in1만능케이블 inCharge6',), ('국민한복 프로젝트 흥과 함께 버무려진 우리 옷 한류 프로젝트',))
========== title ==========

======= selected_word =======
['캠핑', '일리', '한복', '바지', '피부', '운동', '변화', '팬츠', '국민', '슬랙스', '프로젝트']
======= selected_word =======

======= category list =======
['여행·레저', '여행·레저', '패션', '패션', '여행·레저', '뷰티', '여행·레저', '뷰티', '여행·레저', '패션', '패션', '패션']
======= category list =======



  0%|          | 0/12 [00:00<?, ?it/s]

key word: 캠핑 여행·레저

화목

부뚜막

크리스탈

난로

테이블

key word: 일리 여행·레저

입지

골덴

테크니컬

key word: 한복 패션

양단

저고리

버선

두루마기

리버시

key word: 바지 패션

입지

문공

슬랙스

나빌레

팬츠

key word: 바지 여행·레저

입지

문공

슬랙스

나빌레

팬츠

key word: 피부 뷰티

도화지

가려움

앰플

진정

알란

key word: 운동 여행·레저

키네

관성

헬린

스트레칭

key word: 변화 뷰티

해삼

교사

팬데믹

토트

앰플

key word: 팬츠 여행·레저

조거

입지

와이드

테이

퍼드

key word: 국민 패션

아노락

타월

딥컬렉티브

빈센트

땅콩

key word: 슬랙스 패션

퍼드

문공

테이

자시

입지

key word: 프로젝트 패션

사막

투닥

토닥

상생

가족사진


('tumblbug', '패션', '시원한 기능성 하프슬랙스 반바지 T.P.O고민 끝!')
('tumblbug', '패션', '뽀글뽀글 저고리 점퍼뽀고리와 캥거루골덴 바지')
('tumblbug', '패션', '전통에 스타일을 더하다세상에 없던 위드랑 버선 Acc')
('tumblbug', '패션', '누구나 입을수 있지만 아무나 입지 않는 겨울 점퍼 폴')
('wadiz', '뷰티', '진정 마름 가려움 없는 피부 by 청정 제주 허벌케어')
('wadiz', '뷰티', '1000분 지속 민감이들도 하루 종일 만족할 수 있다고? 알란토시카')
('wadiz', '뷰티', '수분크림임상 효과 입증피부건조가려움! 참지 말고 BM크림 바르세요')
('wadiz', '뷰티', '4만원대 LED 디바이스이중기능성 유산균 앰플의 가성비 꿀조합!')
('wadiz', '뷰티', '모공과 피부진정?이거 하나면 끝!!! 구아이아줄렌+센텔라의 환상궁합')
('wadiz', '뷰티', '평점 4.9 앵콜 민감피부 고민종결 붉은기까지 알란토

In [7]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='wdta2181',db='test', charset='utf8')

curs = conn.cursor()

username = "붜니"
sql = "select distinct trim(I.title) from test.user_info I, test.crawl C where username like '%s' and C.title = I.title;"%username

curs.execute(sql)
sent = curs.fetchall()

curs.execute(sql)
title_with_category = curs.fetchall()

print("========== title ==========")
print(title_with_category)
print("========== title ==========\n")

noun_set = set(okt.nouns(str(sent)))
selected_word = []

sent_list = list(sent)
sent_list2 = []
for i in sent_list:
    sent_list2.append(list(i))

print("======= selected_word =======")
#selected_word : tokenized word from user's funding title list
for i in noun_set:
    if i in tfidf_dict:
        selected_word.append(i)
        continue
print(selected_word) 
print("======= selected_word =======\n")

final_word_list = []
category_list=[]

#final_word_list : selected words
print("======= category list =======")
for i in selected_word:
    sql3 = "select distinct I.title, C.category from test.user_info I, test.crawl C where username like '{username}' and C.title = I.title and I.title like '%%{word}%%';".format(username=username, word=i);
    curs.execute(sql3)
    rows = curs.fetchall()
    
    for j in rows:
        if i in j[0]:
            final_word_list.append(i)
            category_list.append(j[1])        
print(category_list)
print("======= category list =======\n")  

model = word2vec.Word2Vec.load("NaverMovie29.model")
word_len = len(final_word_list)
category_len = len(category_list)

final_projects=[]
similar_word_list=[]

for i in tqdm(range(word_len)):
    print("key word: "+final_word_list[i], category_list[i])
    print()
    similar_word=model.wv.most_similar(positive=[final_word_list[i]],topn=5)
    for j in similar_word:
        if len(j[0])==1:
            continue
        else:
            print(j[0])
            sql = "select DISTINCT pagename, category, trim(title) from test.crawl where title like '%%%s%%'" %j[0]
            curs.execute(sql)
            words = curs.fetchall()
            conn.commit()
            #print(words)
            length = len(words)
            if length>3:
                length = 3
            #print(length)
            for k in range(0,length):
                tmp=[]
                tmp.append(''.join(list(words[k][2])))
                if words[k][1] == category_list[i] and tmp not in sent_list2 :
                    final_projects.append(words[k])
        print()   


final_projects_set = set(final_projects)
#for i in final_projects_set:
#    print(i)
#print(final_projects_set)

final_projects_set = list(final_projects_set)
fianl_projects_set = final_projects_set.sort(key = lambda x: [x[0], x[1]])
print()
for i in final_projects_set:
    print(i)
print('개수 : ', len(final_projects_set))

conn.close()

========== title ==========
(('4억앵콜10만원 중반 국민코트 진화된 구스 캐시미어 핸드메이드 코트',), ('1억 여성 드로즈는 얼마나 편할지 궁금하시죠? 기술력이 다른 톤포투',), ('앵콜펀딩 대서양 보습원료로 만든 해초시트! 저자극 보습진정 마스크팩',), ('국가대표가 선택한 양면 벤치코트과연 뭐가 다르길래? 구경은 공짜에요',), ('4차앵콜 유산균이 뭐길래? 피부 속 생태계를 유지할 수 있을까요?',), ('평점 4.9 앵콜 민감피부 고민종결 붉은기까지 알란토시카가 뭔데?',), ('3개임상실험 에센스론 부족해서 시트에도 넣었어요 내볼탱탱볼마스크팩',), ('9천원대 엉덩이 굴곡에 찰  떡 ! 인체 밀착형 웨이비 팬티',), ('리코브모든 노하우를 담아낸 모두가 입고싶고 어울리는 모.두.팬.츠',), ('Y존 냄새분비물뿌리는 생 유산균으로 도움받아요!',), ('열손가락 스마트터치4만원대 5가지 컬러의 리얼 양가죽 장갑',))
========== title ==========

======= selected_word =======
['모든', '냄새', '원료', '국민', '유지', '양면', '코트', '핸드', '드로즈', '웨이', '컬러', '메이드', '기술', '자극', '보습', '노하우', '캐시미어', '피부', '여성', '마스크', '에센스', '스마트', '고민', '장갑', '팬티', '리얼', '가죽', '터치', '구스', '유산균', '다른', '가지', '시트', '진정']
======= selected_word =======

======= category list =======
['패션', '뷰티', '뷰티', '패션', '뷰티', '패션', '패션', '패션', '패션', '패션', '패션', '패션', '패션', '패션', '뷰티', '뷰티', '패션', '패션', '뷰티', '뷰티', '패션', '뷰티', '뷰티', '뷰티', '패션', '뷰티', '패션', '패션', '패션', '패

  0%|          | 0/39 [00:00<?, ?it/s]

key word: 모든 패션

클리셰

필레

우화

key word: 냄새 뷰티

알콜

마밍스

분비물

리운스

헤파

key word: 원료 뷰티

엄선

해초

대서양

등급

프로폴리스

key word: 국민 패션

아노락

타월

딥컬렉티브

빈센트

땅콩

key word: 유지 뷰티

포그

해동

구강

선수

계단

key word: 양면 패션

벤치

구경

스케줄

공짜

양단

key word: 코트 패션

히팅

맥시

폴로

노블레스

체스터필드

key word: 코트 패션

히팅

맥시

폴로

노블레스

체스터필드

key word: 핸드 패션

메이드

시크릿

아인

생초콜릿

크리스

key word: 드로즈 패션

포투

얼마나

성용

그랑프리

key word: 웨이 패션

로퍼

버킷

key word: 컬러 패션

포일

가공

링북

무염

불교

key word: 메이드 패션

핸드

크리스

시크릿

노블레스

매틱

key word: 기술 패션

얼마나

포투

공예

지식

드로즈

key word: 자극 뷰티

해초

대서양

강력

참당귀

key word: 보습 뷰티

대서양

립밤

고정

해초

장벽

key word: 노하우 패션

미드

실전

베이비

쇼핑몰

꿀팁

key word: 캐시미어 패션

제일모직

체스터필드

나일론

타스마니아

key word: 피부 뷰티

도화지

가려움

앰플

진정

알란

key word: 피부 뷰티

도화지

가려움

앰플

진정

알란

key word: 여성 패션

트렁크

위인

비혼

나혜석

서사

key word: 마스크 뷰티

통증

그래핀

스트랩

구리

key word: 마스크 뷰티

통증

그래핀

스트랩

구리

key word: 에센스 뷰티

어성초

사표

히알루론산

유효

생기

key word: 스마트 패션

측정

교통카드

프로젝터

진공청소기

key word: 고민 뷰티

안구건조증

장아찌

해결

알란

